In [3]:
from Tkinter import *
import tkFileDialog
import sys

In [ ]:
# This looks great!
# http://stackoverflow.com/questions/36604900/redirect-stdout-to-tkinter-text-widget


# http://stackoverflow.com/questions/14883648/redirect-output-from-python-logger-to-tkinter-widget

In [4]:
class myApp(object):
    def __init__(self, master, portfolioURL = 'https ://drive.google.com/example/abc123xyz987'):
        self.master = master
        master.title('Student Portfolio Populator')
        
        self.msgReqURL = 'Paste Google Drive URL for Student Portfolio below:'
        self.msgOKURL = 'Student Portfolio URL accepted:'
        self.message = self.msgReqURL
        
        self.portfolioURL = portfolioURL #'https://drive.google.com/'
        self.studentFile = None
        self.studentData = None
        
        self.counter = 0
        
        self.file_opt = options = {}
        options['defaultextension'] = '*.csv'
        options['filetypes'] = [('All', '.py')]
        options['parent'] = root
        
        
        # set window size
        self.master.geometry('800x400+30+30')
#         self.label = Label(self.master, text = 'Paste URL for Student Portfolio')
        
        self.labelURL_text = StringVar()
        self.labelURL_text.set(self.message)
        self.labelURL = Label(self.master, textvariable = self.labelURL_text, font = "TkDefaultFont 0 bold")
 
        self.labelStuFileRow_text = StringVar()
        self.labelStuFileRow_text.set('Student File: ')
        self.labelStuFileRow = Label(self.master, textvariable = self.labelStuFileRow_text, 
                                     font = "TkDefaultFont 0 bold")
        
        self.labelStuFile_text = StringVar()
        self.labelStuFile_text.set('None selected')
        self.labelStuFile = Label(self.master, textvariable = self.labelStuFile_text)
    
        self.submit_button = Button(self.master, text = 'submit', command = self.submit)
        self.quit_button = Button(self.master, text = 'quit', command = self.quit)
        self.buttonStFile = Button(self.master, text = 'set student data file', command = self.studentFilePicker)
        
#         vcmd = master.register(self.validate) # wrapper for command
        self.entryURL = Entry(self.master, width = 60, validate='key') #, validatecommand=(vcmd, '%P'))
        self.entryURL.insert(END, self.portfolioURL)

        self.labelURL.grid(row = 0, columnspan = 2, sticky=W)        
        self.entryURL.grid(row = 2, column = 0, columnspan = 2, sticky = W)
        self.labelStuFileRow.grid(row = 3, column =0, sticky = W)
        self.labelStuFile.grid(row = 3, column = 1, sticky = W)
        self.buttonStFile.grid(row = 4, column = 0, sticky = W)
        self.submit_button.grid(row = 5, column = 0, sticky = W)
        self.quit_button.grid(row = 6, column = 0, sticky = W)  
        
        self.master.deiconify()
        
    def studentFilePicker(self):
        try:
            self.studentFile = tkFileDialog.askopenfile(mode='r', title='Choose exported student data')
        except (IOError, OSError) as e:
            print e
            return(False)
        
        
        try:
            self.studentData = self.studentFile.read()
            self.labelStuFile_text.set(self.studentFile.name)
            print self.studentData
        except AttributeError:
            pass
        
        return(True)
    
    def spawnNew(self):
        win = Toplevel()
        win.title('stuff')
        message = Message(win, text ="some more stuff")
        message.pack()


        
    def submit(self):
        self.portfolioURL = self.entryURL.get()
        print self.portfolioURL
        if len(self.portfolioURL)==0:
            self.message = self.msgReqURL
        else:
            # add a check for a valid url here
            self.message = self.msgOKURL
        
        self.labelURL_text.set(self.message)
        
        # spawn a text window for the output from the folder creation application
        #         self.spawnNew()
        
    def greet(self):
        print('hello world')
        
    def quit(self):
        self.master.destroy()

root = Tk()
my_gui = myApp(root)
root.mainloop()

In [ ]:
from Tkinter import Tk

In [ ]:
Text()

In [ ]:
my_gui.labelURL.cget()